# Context Collector - Context per Term (CPT)

In [1]:
import os
from pathlib import Path
import shutil

In [ ]:
#vec_root = Path("/home/max/Corpora/flashback-pol-time/yearly/contexts/vectors/")
#ctx_root = Path("/home/max/Corpora/flashback-pol-time/yearly/contexts/files/")
#vpt_root = Path("/home/max/Corpora/flashback-pol-time/yearly/contexts_per_term/vectors/")
#xpt_root = Path("/home/max/Corpora/flashback-pol-time/yearly/contexts_per_term/files/")

In [2]:
vec_root = Path("/home/max/Corpora/familjeliv-smh-time/yearly/contexts/vectors/")
ctx_root = Path("/home/max/Corpora/familjeliv-smh-time/yearly/contexts/files/")
vpt_root = Path("/home/max/Corpora/familjeliv-smh-time/yearly/contexts_per_term/vectors/")
xpt_root = Path("/home/max/Corpora/familjeliv-smh-time/yearly/contexts_per_term/files/")

In [3]:
# find terms
terms = set()
for directory in os.listdir(vec_root):
    for file in os.listdir(vec_root / directory):
        with open(vec_root / directory / file) as f:
            for line in f:
                term, _ = tuple(line.strip("\n").split("\t"))
                terms.add(term)
    break

In [4]:
terms

{'A1_globalistisk',
 'N1C_berikareX',
 'N1C_globalistX',
 'N1C_kulturberikarX',
 'N1C_återvandringsX',
 'N1_berikare',
 'N1_förortsgäng',
 'N1_globalist',
 'N1_kulturberikare',
 'N1_återvandring',
 'N2C_återvandrarX',
 'N2_återvandrare',
 'P1_ordning_och_reda_i_flyktingpolitiken',
 'V1_berika',
 'V1_hjälpa_på_plats',
 'V1_kulturberika',
 'V1_återvandra',
 'V2_hjälpa_X_på_plats',
 'X_berika',
 'X_globalist',
 'X_hjälpa betydligt fler på plats',
 'X_hjälpa klimatproblemen " på plats',
 'X_hjälpa mig och beskriva lite om hur det går till från att ni får ett larm till personen är i säkerhet , eller i värsta fall ni hämtar någon som är så illa tilltygad att den förklaras död på plats',
 'X_hjälpa oss att komma in i samhället alla är upptagna att ta emot fler fler och fler istället för att hjälpa på plats',
 'X_hjälpa på plats',
 'X_hjälpanläggningar på plats',
 'X_hjälparbetare på plats',
 'X_hjälparbetare till andra länder och donerar mycket till att bygga skolor etc på plats',
 'X_hjälpar

In [5]:
selected_terms = [
 'A1_globalistisk',
 'N1C_berikareX',
 'N1C_förortsgängX',
 'N1C_globalistX',
 'N1C_kulturberikarX',
 'N1C_återvandringsX',
 'N1_berikare',
 'N1_förortsgäng',
 'N1_globalist',
 'N1_kulturberikare',
 'N1_återvandring',
 'N2C_återvandrarX',
 'N2_återvandrare',
 'P1_ordning_och_reda_i_flyktingpolitiken',
 'P1_självständig_utrikespolitik',
 'V1_berika',
 'V1_hjälpa_på_plats',
 'V1_kulturberika',
 'V1_återvandra',
 'V2_hjälpa_X_på_plats'
]

In [6]:
merger = {
    "N1C_berikareX": "N1_berikare",
    "N1C_förortsgängX": "N1_förortsgäng",
    "N1C_globalistX": "N1_globalist",
    "N1C_kulturberikarX": "N1_kulturberikare",
    "N1C_återvandringsX": "N1_återvandring",
    "N2C_återvandrarX": "N2_återvandrare",
}

In [7]:
# restrict terms
merge = True
if merge:
    selected_terms = [term for term in selected_terms if term not in merger]
selected_terms

['A1_globalistisk',
 'N1_berikare',
 'N1_förortsgäng',
 'N1_globalist',
 'N1_kulturberikare',
 'N1_återvandring',
 'N2_återvandrare',
 'P1_ordning_och_reda_i_flyktingpolitiken',
 'P1_självständig_utrikespolitik',
 'V1_berika',
 'V1_hjälpa_på_plats',
 'V1_kulturberika',
 'V1_återvandra',
 'V2_hjälpa_X_på_plats']

In [8]:
# create files for vectors per term
for directory in os.listdir(vec_root): # fb_nli etc.
    os.mkdir(vpt_root / directory)
    for x in selected_terms:
        file = open(vpt_root / directory / f"{x}.txt", mode = "x")
        file.close()

In [9]:
# create files for contexts (words) per term
for x in selected_terms:
    file = open(xpt_root / f"{x}.txt", mode = "x")
    file.close()

In [10]:
# count 
total = 0
for directory in os.listdir(vec_root):
    for file in os.listdir(vec_root / directory):
        with open(vec_root / directory / file) as f:
            for line in f:
                term = line.split("\t")[0]
                
                if merge:
                    if term in merger:
                        term = merger[term]
                
                if term in selected_terms:
                    total += 1
    break

In [11]:
total

4281

In [12]:
# build context per term - vectors (CPT)
for directory in os.listdir(vec_root):
    count = 0
    print(directory)
    for file in sorted(os.listdir(vec_root / directory)):
        year = file.replace(".txt", "")
        with open(vec_root / directory / file) as f: 
            for line in f:
                term, vector = tuple(line.strip("\n").split("\t"))
                #context = context.strip("\n")

                if merge:
                    if term in merger:
                        term = merger[term]                
                
                if term not in selected_terms:
                    continue
                filename = f"{term}.txt"
                
                with open(vpt_root / directory / filename, mode="a") as g:
                    g.write(f"{year}\t{vector}\n")
                print(round((count / total)*100, 3), "%", end="\r")
                count += 1
    print()

sentence-bert-swedish-cased
99.977 %


In [13]:
# build context per term - words (XPT) 
count = 0
for file in sorted(os.listdir(ctx_root)):
    year = file.replace(".txt", "")
    with open(ctx_root / file) as f: 
        for line in f:
            terms, n_typ, context = tuple(line.strip("\n").split("\t"))
            #context = context.strip("\n")
            
            for term in terms.split("; "):

                if merge:
                    if term in merger:
                        term = merger[term]                

                if term not in selected_terms:
                    continue

                filename = f"{term}.txt"

                with open(xpt_root / filename, mode="a") as g:
                    g.write(f"{year}\t{context}\n")
                print(round((count / total)*100, 3), "%", end="\r")
                count += 1
print()
print(count)

99.977 %
4281
